In [1]:
# Instalar pyodbc
# import sys
# !{sys.executable} -m pip install pyodbc
# import sys
# !{sys.executable} -m pip install openpyxl
# !{sys.executable} -m pip install --upgrade seaborn
# !{sys.executable} -m pip install --upgrade matplotlib

In [2]:
#Librería pyodbc
import pyodbc
import pandas as pd
import numpy as np
import openpyxl

# Conexión al Access
# Creo función 
def py_conn():
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=C:\\Users\\MEC\\OneDrive - BCRA\\General\\GH\\clustering.accdb;'
        )
    conn = pyodbc.connect(conn_str)

    return conn


def py_conn_tables(cursor):
    for table_info in cursor.tables(tableType='TABLE'):
        print(table_info.table_name)

In [3]:
# Consulta tabla indicadores
conn = py_conn()
data = pd.read_sql_query("SELECT * FROM DATA", conn)
conn.close()
# Veo la tabla que me trajo
data_full = data
print(data)

C:\Users\MEC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


        EENTIDAD CODIND     FECHAS         IND
0          00262    C31 2019-03-01  702.188401
1          00262    C31 2019-04-01  689.433633
2          00262    C31 2019-05-01  660.587483
3          00262    C31 2019-06-01  606.182058
4          00262    C31 2019-07-01  540.094705
...          ...    ...        ...         ...
2278404        8    RG9 2021-07-01   -7.141944
2278405        8    RG9 2021-08-01   -7.100442
2278406        8    RG9 2021-09-01   -7.357433
2278407        8    RG9 2021-10-01   -7.415377
2278408        8    RG9 2021-11-01   -7.303206

[2278409 rows x 4 columns]


In [4]:
# Elimino sistema, ghs y gh8
gh = ['1', '2', '3', '4', '5', '6', '7', '8', '00', '10', '11',
		'44099', '44098', '44096', '44095', '44088', '44093', '44092', '44090', '00340', '44059', '44094'
		]
data= data[~data['EENTIDAD'].isin(gh)]

In [5]:
# Elimino indicadores que no necesito
ind_cod = [
          'RG2', 'RG1', 'E2', 'E1', 'RAPM',
          'A6', 'AG1', 'A2', 'A21',
          'L1', 'L8',
          'C1', 'C14', 'C2', 
          'LS1', 'LS2', 'MS1'
		]
data = data[data['CODIND'].isin(ind_cod)]


Empty DataFrame
Columns: [EENTIDAD, CODIND, FECHAS, IND]
Index: []


In [6]:
# Datos desde 2013
data = data[data['FECHAS'].dt.strftime('%Y') >= '2013']
data_full = data_full[data_full['FECHAS'].dt.strftime('%Y') >= '2013']
print(data)

        EENTIDAD CODIND     FECHAS        IND
29393      00005     A2 2013-01-01  20.070590
29394      00005     A2 2013-02-01  20.675030
29395      00005     A2 2013-03-01  21.914390
29396      00005     A2 2013-04-01  30.937750
29397      00005     A2 2013-05-01  33.333333
...          ...    ...        ...        ...
2230309    65203    RG2 2021-07-01  62.946652
2230310    65203    RG2 2021-08-01  66.162508
2230311    65203    RG2 2021-09-01  66.941748
2230312    65203    RG2 2021-10-01  67.309958
2230313    65203    RG2 2021-11-01  68.432335

[111758 rows x 4 columns]


In [7]:
# Elimino entidades NO vigentes
conn = py_conn()
entidad = pd.read_sql_query("SELECT * FROM ENTIDAD", conn)
conn.close()
vigente = entidad['EENTIDAD'][entidad['VIGENTE']==True]
# print(vigente)
data = data[data['EENTIDAD'].isin(vigente)]
data_full = data_full[data_full['EENTIDAD'].isin(vigente)]

C:\Users\MEC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
# Cuantos valores nulos hay
print(f'Cantidad de valores nulos por columna:\n{data.isna().sum()}') # Chequear RAPM

Cantidad de valores nulos por columna:
EENTIDAD    0
CODIND      0
FECHAS      0
IND         0
dtype: int64


In [9]:
# Conjunto de indicadores con valore no nulos
data = data[data.isna()==False]

# Creo la columna de año, saco promedio de cada diciembre
data['%Y'] = data['FECHAS'].dt.strftime('%Y')
data_dic_mean = data
data_dic_mean['IND'].replace(0, np.nan)
data_dic_mean = data_dic_mean.groupby(['EENTIDAD', 'CODIND', '%Y']).mean()

# Tabla con promedios totales
data_mean = data.replace(0, np.nan)
data_mean = data.drop(columns=['FECHAS']).groupby(['EENTIDAD', 'CODIND'], as_index=False).mean()
# data_mean = data_mean.replace(0, np.nan)
# print(data_mean)

print(f'Cantidad de valores nulos por columna:\n{data.isna().sum()}')
print(f'Cantidad de valores nulos por columna:\n{data_dic_mean.isna().sum()}')
print(f'Cantidad de valores nulos por columna:\n{data_mean.isna().sum()}')

data_mean = data_mean.reset_index()
data_dic_mean = data_dic_mean.reset_index()

Cantidad de valores nulos por columna:
EENTIDAD    0
CODIND      0
FECHAS      0
IND         0
%Y          0
dtype: int64
Cantidad de valores nulos por columna:
IND    0
dtype: int64
Cantidad de valores nulos por columna:
EENTIDAD    0
CODIND      0
IND         0
dtype: int64


In [10]:
import os
from openpyxl import load_workbook

def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    @param filename: File path or existing ExcelWriter
                     (Example: '/path/to/file.xlsx')
    @param df: DataFrame to save to workbook
    @param sheet_name: Name of sheet which will contain DataFrame.
                       (default: 'Sheet1')
    @param startrow: upper left cell row to dump data frame.
                     Per default (startrow=None) calculate the last row
                     in the existing DF and write to the next row...
    @param truncate_sheet: truncate (remove and recreate) [sheet_name]
                           before writing DataFrame to Excel file
    @param to_excel_kwargs: arguments which will be passed to `DataFrame.to_excel()`
                            [can be a dictionary]
    @return: None

    Usage examples:

    >>> append_df_to_excel('d:/temp/test.xlsx', df)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, header=None, index=False)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, sheet_name='Sheet2',
                           index=False)

    >>> append_df_to_excel('d:/temp/test.xlsx', df, sheet_name='Sheet2', 
                           index=False, startrow=25)

    (c) [MaxU](https://stackoverflow.com/users/5741205/maxu?tab=profile)
    """
    # Excel file doesn't exist - saving and exiting
    if not os.path.isfile(filename):
        df.to_excel(
            filename,
            sheet_name=sheet_name, 
            startrow=startrow if startrow is not None else 0, 
            **to_excel_kwargs)
        return
    
    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl', mode='a')

    # try to open an existing workbook
    writer.book = load_workbook(filename)
    
    # get the last row in the existing Excel sheet
    # if it was not specified explicitly
    if startrow is None and sheet_name in writer.book.sheetnames:
        startrow = writer.book[sheet_name].max_row

    # truncate sheet
    if truncate_sheet and sheet_name in writer.book.sheetnames:
        # index of [sheet_name] sheet
        idx = writer.book.sheetnames.index(sheet_name)
        # remove [sheet_name]
        writer.book.remove(writer.book.worksheets[idx])
        # create an empty sheet [sheet_name] using old index
        writer.book.create_sheet(sheet_name, idx)
    
    # copy existing sheets
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

In [11]:
# Guardo excel con tablas de datos individuales y promedios

ind = list(set(data_dic_mean['CODIND']))
periodo = sorted(list(set(data_dic_mean['%Y'])))
ind_per = [(i, p) for i in ind for p in periodo]

with pd.ExcelWriter('output/data_mean.xlsx') as writer:  
    data.to_excel(writer, sheet_name='data')
    data_dic_mean.to_excel(writer, sheet_name='data_dic_mean')
    data_mean.to_excel(writer, sheet_name='data_mean')
    for ind, per in ind_per:
        if per == min(periodo):
            data_xlsx = data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()

        else:
            data_xlsx = pd.concat([data_xlsx, 
            data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()], axis=1)

        if per == max(periodo):
            data_xlsx.to_excel(writer, sheet_name=str(ind), index=False)

C:\Users\MEC\AppData\Local\Temp\ipykernel_9040\2556071299.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_xlsx = data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()
C:\Users\MEC\AppData\Local\Temp\ipykernel_9040\2556071299.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()], axis=1)
C:\Users\MEC\AppData\Local\Temp\ipykernel_9040\2556071299.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()], axis=1)
C:\Users\MEC\AppData\Local\Temp\ipykernel_9040\2556071299.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()], axis=1)
C:\Users\MEC\AppData\Local\Temp\ipykernel_9040\255

In [12]:
# Creo datos para histogramas en Excel

'''ind = list(set(data_dic_mean['CODIND']))
periodo = sorted(list(set(data_dic_mean['%Y'])))

ind_per = [(i, p) for i in ind for p in periodo]

with pd.ExcelWriter('output/data.xlsx') as writer: 
    for ind, per in ind_per:
        if per == min(periodo):
            data_xlsx = data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()

        else:
            data_xlsx = pd.concat([data_xlsx, 
            data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()], axis=1)

        if per == max(periodo):
            data_xlsx.to_excel(writer, sheet_name=str(ind), index=False)
            append_df_to_excel('output/data.xlsx', data_xlsx, sheet_name=ind)'''

"ind = list(set(data_dic_mean['CODIND']))\nperiodo = sorted(list(set(data_dic_mean['%Y'])))\n\nind_per = [(i, p) for i in ind for p in periodo]\n\nwith pd.ExcelWriter('output/data.xlsx') as writer: \n    for ind, per in ind_per:\n        if per == min(periodo):\n            data_xlsx = data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()\n\n        else:\n            data_xlsx = pd.concat([data_xlsx, \n            data_dic_mean[data_dic_mean['CODIND']==ind][data_dic_mean['%Y']==per].reset_index()], axis=1)\n\n        if per == max(periodo):\n            data_xlsx.to_excel(writer, sheet_name=str(ind), index=False)\n            append_df_to_excel('output/data.xlsx', data_xlsx, sheet_name=ind)"

In [13]:
'''# Creo tabla pivot serie de tiempo y grafico
# from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

indicador = list(set(data_dic_mean['CODIND']))
periodo = sorted(list(set(data_dic_mean['%Y'])))
# ind_per = [(i, p) for i in indicador for p in periodo]
# ind = 'A21'
for ind in indicador: 
    data_dic_mean_pivot = data_dic_mean[data_dic_mean['CODIND']==ind].pivot(index='EENTIDAD', columns='%Y', values='IND').reset_index()
    data_dic_mean_pivot = data_dic_mean_pivot.set_index('EENTIDAD')
    data_dic_mean_pivot = data_dic_mean_pivot.fillna(0.0)

    # Método Simple:
    # bins = math.ceil(math.sqrt(data_dic_mean_pivot.shape[0] * data_dic_mean_pivot.shape[1]))
    # bin_width = (max(data_dic_mean_pivot.max()) - min(data_dic_mean_pivot.min())) / bins

    # Método Regla Sturge:
    bins = int(np.ceil(np.log2(data_dic_mean_pivot.shape[0] * data_dic_mean_pivot.shape[1])) + 1)

    # Método Regla -----:

    # Puede que ande lento por cuestión de bins

    plt.style.use('ggplot')
    # plt.tight_layout()
    # plt.suptitle(f'Distribución {ind} promedio diciembre desde {min(periodo)} hasta {max(periodo)}')
    sns.pairplot(data=data_dic_mean_pivot, diag_kws = {"bins": bins})
    plt.savefig(f'graph/hist-scatter-{ind}-{min(periodo)}-{max(periodo)}.jpg')
    plt.clf() # Clean parirplot figure from sns '''

'# Creo tabla pivot serie de tiempo y grafico\n# from IPython.display import Image\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\nindicador = list(set(data_dic_mean[\'CODIND\']))\nperiodo = sorted(list(set(data_dic_mean[\'%Y\'])))\n# ind_per = [(i, p) for i in indicador for p in periodo]\n# ind = \'A21\'\nfor ind in indicador: \n    data_dic_mean_pivot = data_dic_mean[data_dic_mean[\'CODIND\']==ind].pivot(index=\'EENTIDAD\', columns=\'%Y\', values=\'IND\').reset_index()\n    data_dic_mean_pivot = data_dic_mean_pivot.set_index(\'EENTIDAD\')\n    data_dic_mean_pivot = data_dic_mean_pivot.fillna(0.0)\n\n    # Método Simple:\n    # bins = math.ceil(math.sqrt(data_dic_mean_pivot.shape[0] * data_dic_mean_pivot.shape[1]))\n    # bin_width = (max(data_dic_mean_pivot.max()) - min(data_dic_mean_pivot.min())) / bins\n\n    # Método Regla Sturge:\n    bins = int(np.ceil(np.log2(data_dic_mean_pivot.shape[0] * data_dic_mean_pivot.shape[1])) + 1)\n\n    # Método Regla -----:\n\n    # Pued

In [14]:
# Creo tabla pivot serie de tiempo
data_mean_pivot = data_mean.pivot(index='EENTIDAD', columns='CODIND', values='IND').reset_index()
data_mean_pivot = data_mean_pivot.set_index('EENTIDAD')
print(data_mean_pivot.isna().sum())
data_mean_pivot.fillna(0.0)

'''# Método Regla Sturge:
bins = int(np.ceil(np.log2(data_dic_mean_pivot.shape[0] * data_dic_mean_pivot.shape[1])) + 1)

# Método Regla -----:

# Puede que ande lento por cuestión de bins

plt.style.use('ggplot')
# plt.tight_layout()
# plt.suptitle(f'Distribución {ind} promedio diciembre desde {min(periodo)} hasta {max(periodo)}')
sns.pairplot(data=data_mean_pivot, diag_kws = {"bins": bins})
plt.savefig(f'graph/hist-scatter-{min(periodo)}-{max(periodo)}.jpg')
plt.clf() # Clean parirplot figure from sns '''

CODIND
A2      0
A21     3
AG1     2
C1      0
C14     0
C2      0
E1      0
E2      0
L1      0
L8      0
LS1     0
LS2     1
MS1     0
RAPM    0
RG1     0
RG2     0
dtype: int64


'# Método Regla Sturge:\nbins = int(np.ceil(np.log2(data_dic_mean_pivot.shape[0] * data_dic_mean_pivot.shape[1])) + 1)\n\n# Método Regla -----:\n\n# Puede que ande lento por cuestión de bins\n\nplt.style.use(\'ggplot\')\n# plt.tight_layout()\n# plt.suptitle(f\'Distribución {ind} promedio diciembre desde {min(periodo)} hasta {max(periodo)}\')\nsns.pairplot(data=data_mean_pivot, diag_kws = {"bins": bins})\nplt.savefig(f\'graph/hist-scatter-{min(periodo)}-{max(periodo)}.jpg\')\nplt.clf() # Clean parirplot figure from sns '

In [15]:
'''indicador = list(set(data_dic_mean['CODIND']))
periodo = sorted(list(set(data_dic_mean['%Y'])))
entidad = sorted(list(set(data_dic_mean['EENTIDAD'])))
n_ind = len(indicador)
n_per = len(periodo)
n_ent = len(entidad)
ent_ind = sorted([(e, i) for e in entidad for i in indicador])
a = n_ind  # number of rows
b = 3  # number of columns
c = 1  # initialize plot counter

# print(data_mean_pivot.loc[data_mean_pivot.index[0]])
# print(data_dic_mean[data_dic_mean['EENTIDAD']=='00007'][data_dic_mean['CODIND']==ind])
# print(ent_ind[0])

fig = plt.figure(figsize=(20,100))
fig.tight_layout
# plt.suptitle(f'{ent} - {ind} ({min(periodo)} hasta {max(periodo)})')

for ent in entidad:
    c = 1
    
    for ind in indicador:
        print(ent, ind)
        data_plt = data_dic_mean[data_dic_mean['EENTIDAD']==ent][data_dic_mean['CODIND']==ind]
        
        plt.subplot(a, b, c)
        plt.title(f'Dist. Medias Anuales')
        plt.xlabel(f'{ent} - {ind}')
        sns.distplot(data_plt['IND'], bins=bins)
        c = c + 1

        plt.subplot(a, b, c)
        plt.title(f'Boxplot Medias Anuales')
        plt.xlabel(f'{ent} - {ind}')
        plt.boxplot(x=data_plt['IND'])
        c = c + 1

        plt.subplot(a, b, c)
        plt.title(f'Evol. Medias Anuales')
        plt.xlabel(f'{ent} - {ind}')
        sns.lineplot(data=data_plt, x=data_plt['%Y'], y=data_plt['IND'])
        c = c + 1

    plt.savefig(f'graph/analisis-indicadores-{ent}-{min(periodo)}-{max(periodo)}.jpg')
    plt.clf() # Clean parirplot figure from sns 
    # plt.show()'''

"indicador = list(set(data_dic_mean['CODIND']))\nperiodo = sorted(list(set(data_dic_mean['%Y'])))\nentidad = sorted(list(set(data_dic_mean['EENTIDAD'])))\nn_ind = len(indicador)\nn_per = len(periodo)\nn_ent = len(entidad)\nent_ind = sorted([(e, i) for e in entidad for i in indicador])\na = n_ind  # number of rows\nb = 3  # number of columns\nc = 1  # initialize plot counter\n\n# print(data_mean_pivot.loc[data_mean_pivot.index[0]])\n# print(data_dic_mean[data_dic_mean['EENTIDAD']=='00007'][data_dic_mean['CODIND']==ind])\n# print(ent_ind[0])\n\nfig = plt.figure(figsize=(20,100))\nfig.tight_layout\n# plt.suptitle(f'{ent} - {ind} ({min(periodo)} hasta {max(periodo)})')\n\nfor ent in entidad:\n    c = 1\n    \n    for ind in indicador:\n        print(ent, ind)\n        data_plt = data_dic_mean[data_dic_mean['EENTIDAD']==ent][data_dic_mean['CODIND']==ind]\n        \n        plt.subplot(a, b, c)\n        plt.title(f'Dist. Medias Anuales')\n        plt.xlabel(f'{ent} - {ind}')\n        sns.dis

In [16]:
data_mean_pivot_describe = data_mean_pivot.describe()
data_mean_pivot_describe = data_mean_pivot_describe.transpose()
data_mean_pivot_describe = data_mean_pivot_describe.reset_index()

print(data_mean_pivot_describe)

   CODIND  count           mean           std          min        25%  \
0      A2   68.0       8.072969  8.475738e+00     2.250006   4.950129   
1     A21   65.0     -10.337920  1.894048e+01  -117.104961 -12.629326   
2     AG1   66.0      21.276164  1.060360e+01     1.863860  14.142290   
3      C1   68.0       5.947488  3.449192e+00     0.107298   3.219895   
4     C14   68.0  618458.796489  3.488538e+06     0.000000  66.817093   
5      C2   68.0      12.883508  2.122279e+01    -7.913633   1.591412   
6      E1   68.0   29075.204436  2.312638e+05     2.871210   6.405195   
7      E2   68.0      44.474778  7.073031e+01   -83.814146   5.736029   
8      L1   68.0   45733.445428  2.878710e+05    28.092900  39.059352   
9      L8   68.0   43179.251426  2.412474e+05    26.934974  42.832278   
10    LS1   68.0      -2.029558  1.852066e+00    -7.755370  -2.607682   
11    LS2   67.0      -0.255793  1.725177e-01    -0.595633  -0.372146   
12    MS1   68.0    1184.006556  5.284066e+03    -0